In [ ]:
#取路段名
from bs4 import BeautifulSoup
import requests
url = 'http://house.nfu.edu.tw/FJU/acc_info/387.html'
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text,'lxml').select_one('table > tr > td').text
soup

In [ ]:
#抓新北；台北市路段的網址
from bs4 import BeautifulSoup
import requests
import splinter
import json
import time

tool1 = []        #製作jsonarray用

t1 = time.time()  #紀錄時間

url = 'http://house.nfu.edu.tw/protal/all/3.html'
res = requests.get(url)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text,'lxml').select('.readmore > a') 
schoollist = [row.text[2:] for row in soup]   #取得各大學英文代號

for w in range(len(schoollist)):
    URL = 'http://house.nfu.edu.tw/{}/accident.html'.format(schoollist[w])
    browser = splinter.Browser(driver_name='chrome')   #以splinter開google chrome模擬
    browser.visit(URL)
    time.sleep(1)

    try: #寫try except可以讓程式碰到索引超過的時候，依然進行
        for i in range(1,20):
            browser.click_link_by_id('gmapSidebarItem_'+str(i))       #點擊右側欄位
            browser.find_by_id('gmapSidebarItem_'+str(i)).mouse_out() #模擬滑鼠移開
            time.sleep(1)
            element = browser.find_link_by_partial_text('詳細資料')    #抓取連結
            tool1.append(element['href'])                           
            browser.find_link_by_partial_text('詳細資料').mouse_out()  #模擬滑鼠移開

    except Exception as e:
        print(schoollist[w]+'  '+'out of range or no Object')

    print(schoollist[w]+'  done'+'  目前共'+str(len(tool1)))           #目前總共抓了幾個網址
    
    browser.quit()                      #抓完一個學校的物件，瀏覽器要關閉
    
with open('./cloudDANGER/cloudrent_newtaipei_ROAD_link.json','w') as f:   #寫出檔案
    json.dump(tool1,f)    
    
t3 = time.time()  #計算消耗時間
print(t3-t1)

In [ ]:
taipei_postal_code = {'TNUA':112,'CUTE':116,'TCMT':111,'TMUST':114,'USCTP':104,'TPCU':112,'NTCB':100,\
                      'NTUNHS':112,'NTUT':106,'NTNU':106,'NTUST':106,'YM':112,'UTP':100,'PCCU':111,\
                      'TCPA':116,'MCU':111,'NTUE':106,'NCCU':116,'CUST':115,'SLHS':0,'Daan':0,'CCSH':0,\
                      'SAIHS':0,'FHSH':0,'HPSH':0,'DCSH':0,'TFVS':0,'CYHStp':0,'TMU':110}
newtaipei_postal_code = {'AU':251,'HWH':235,'MMC':252,'HFU':223,'FJU':242,'NTPU':237,'MMNMC':112,'SJU':251,\
                         'TKU':251,'HWU':244,'NTUA':220,'JUST':231,'CLUT':220,'TNU':222,'OIT':220,\
                         'MCUT':243,'DLIT':236,'CTCN':231,'LITTP':243}

In [ ]:
with open('./cloudDANGER/cloudrent_newtaipei_ROAD_link.json') as D:
    json_in = D.read()
    json_out = json.loads(json_in)
newtaipei_postal_code[re.findall('[A-Z]{2,4}',json_out[23])[0]]

In [ ]:
#google api 轉經緯度，郵遞區號

import requests


api_url= "https://maps.googleapis.com/maps/api/geocode/json?address={}&key="
def get_location(address):
    URL=api_url.format(address)
    res= requests.get(URL)
    location_dict = res.json()
    lat = location_dict['results'][0]['geometry']['location']['lat']
    lng = location_dict['results'][0]['geometry']['location']['lng']
    cityID = location_dict['results'][0]['address_components'][-1]['short_name']
    res.close()
    return lat,lng,cityID

address = '工專路與華新街143巷路口'
get_location(address)  #tuple

In [ ]:
#新北市路段存檔
from bs4 import BeautifulSoup
import requests
import splinter
import json
import time
import re


def get_location(address):
    URL=api_url.format(address)
    res= requests.get(URL)
    location_dict = res.json()
    lat = location_dict['results'][0]['geometry']['location']['lat']
    lng = location_dict['results'][0]['geometry']['location']['lng']
    cityID = location_dict['results'][0]['address_components'][-1]['short_name']
    res.close()
    return lat,lng,cityID
    
with open('./cloudDANGER/Taipei/cloudrent_taipei_ROAD_link.json') as D:
    json_in = D.read()
    json_out = json.loads(json_in)
    
jsonarray = []   #製作jsonarray用    

t1 = time.time()    
#（三）利用phantomjs讓splinter背景執行---進到各物件的連結，抓取租屋的資訊
from splinter import Browser
executable_path = {'executable_path':'/usr/local/bin/phantomjs'}  #要先安裝好phantomjs。這兩行市基本的程式設定。
browser = Browser('phantomjs', **executable_path)

for i in range(len(json_out)):
    try:
        cloudDANGER = {}  #製作json用

        time.sleep(1)

        res = requests.get(json_out[i])
        res.encoding = 'utf-8'
        soup1 = BeautifulSoup(res.text,'lxml').select_one('table > tr > td').text

        #存網頁連結，物件更新時間（年月日），網頁內容
        cloudDANGER['url'] = json_out[i]
        cloudDANGER['update'] = '2017-08-22'
        cloudDANGER['road'] = soup1
        try:
            api_url= "https://maps.googleapis.com/maps/api/geocode/json?address={}&key="   
            get_location(address =  cloudDANGER['road'])  
       
            time.sleep(2)
            cloudDANGER['cityID'] = get_location(address = cloudDANGER['road'])[2]
            cloudDANGER['lng'] = get_location(address = cloudDANGER['road'])[0]
            cloudDANGER['lat'] = get_location(address = cloudDANGER['road'])[1]
        except:
            cloudDANGER['cityID'] = taipei_postal_code[re.findall('[A-Z]{2,4}',json_out[i])[0]]
            cloudDANGER['lng'] = "NA"
            cloudDANGER['lat'] = "NA"
            

        jsonarray.append(cloudDANGER)   #將一個個物件存進array

        print(len(jsonarray))   #紀錄完成幾筆

        with open('./cloudDANGER/Taipei/\
        Taipei_ROAD_DANGER_{0}.json'.format(i),'w') as F:   #寫出檔案
            json.dump(cloudDANGER,F)

    except:
         print(json_out[i]+'   '+'error')

with open('./cloudDANGER/Taipei/cloudrent_taipei_ROAD_Complete.json','w') as f:   #寫出檔案
    json.dump(jsonarray,f)
    
t2 = time.time()    
print(t2-t1)